# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time
import numpy as np

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.wavemeter import wlm_monitor
from pylabnet.network.client_server import nidaqmx_card
from pylabnet.utils.sweeper import Sweep1D

Enter host and port parameters here:

In [2]:
logger_host = '192.168.0.106'
logger_port = 1386
wavemeter_host = '140.247.189.125'
wavemeter_port = 1477
nidaqmx_host = '140.247.189.125'
nidaqmx_port = 3604

Instantiate all clients

In [3]:
log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
# wm = wlm_monitor.Client(
#     host=wavemeter_host,
#     port=wavemeter_port,
# )
# ni = nidaqmx_card.Client(
#     host=nidaqmx_host,
#     port=nidaqmx_port
# )
exp = Sweep1D(logger=log)

## Experiment

Parameters, setup

In [4]:
f0 = 406.693
f1 = 406.695
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts, reps=4)
# counter = ni.create_timed_counter(
#     counter_channel='Dev1/Ctr0',
#     physical_channel='/Dev1/PFI0'
# )

Experiment script

In [5]:
def experiment(frequency):

    # Set frequency
    # wm.update_parameters([{
    #     'channel': 1, 'setpoint': frequency
    # }])

    # Activate counter
    # ni.start_timed_counter(counter)
    # time.sleep(0.1)

    # Get counts/sec
    # return ni.get_count(counter)*10
    time.sleep(0.02)
    diffusion = 0.0002*np.sin(2*np.pi*time.time()/10)
    return 100 + 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2)
    

In [6]:
exp.configure_experiment(experiment)

In [7]:
exp.set_parameters(reps=100)
exp.run(plot=True)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap', 'uid': '829984d3-338e-4aa8-b265-91d82178e4ce', 'x': [], 'y': […

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap', 'uid': '1dc9b016-e212-40f6-a70e-b948d2182818', 'x': [], 'y': […

## Analyze data

In [9]:
exp.hplot_fwd._y_ar

array([1.])

## Close counter

In [5]:
#ni.close_timed_counter('counter_0')